# 1. Ödev

Bu ödevde herhangi bir şekilde varolan sofistike kütüphaneler kullanılamaz. Sadece rastgele sayı üretmek için kütüphane kullanabilirsiniz. Dokümanları vektörize etmek, uzaklık hesapları yapmak vs. gibi her şey için gereken kodu kendiniz yazmalısınız.

Önemli not: ChatGpt'ye ben de kod yazdırabiliyorum :) ChatGPT'den alınan cevaplar kopya muamelesi görecektir.

Ödeviniz Python 3 ile implement ediniz

**Soru 1** Bu soruda locality sensitive hashing (LSH) metodu yazacaksınız. Metodun çağrımı şu şekildedir: lsh(klasor_adi, k, b, r, similarity_threshold). 
- klasor_adi indeksin kurulacağı dosyaları tutan klasörün adresidir. Klasörün içinde birçok txt dosyası var olduğunu varsayabilirsiniz.
- k 'k-shingle'in k değeridir. 
- b band sayısının değeridir.
- r her banddaki satır (row) sayısıdır. 

Verilen  klasördeki dosyaları okuyarak LSH sistemi kurun. Bunun için her kelime bir token olacaktır. Öncelikle her kelime küçük harflere çevrilmeli ve noktalama işaretleri çıkarılmalı. Eğer k = 2 ise, “Ali okula gitti.” cümlesindeki ‘shingle’lar “ali okula” ve “okula gitti” olmalı. 

MinHash imzaları için 10 tane hash fonksiyonu kullanınız. 

Not: Sorunun cevabını kontrol ederken ben size verilen dosyalar ile aynı formatta olan farklı dosyalar kullanacağım.


In [1]:
def zero_vec(length): # length uzunluğunda 0'lardan oluşan bir vektör döndürür
    vec = []
    for i in range(length):
        vec.append(0)
    return vec

In [182]:
def create_hash_functions(num_hash_functions, num_of_shingles):
    import random
    hash_functions = []
    for i in range(num_hash_functions):
        a = random.randint(1, num_of_shingles-1)
        b = random.randint(1, 10000000)
        if [a, b] not in hash_functions:
            hash_functions.append([a, b])
        else:
            i -= 1
        
    return hash_functions

In [183]:
def transpose_dict(dict):
    matrix = []
    for i in dict.keys():
        matrix.append(dict[i])

    transposed = []
    for i in range(len(matrix[0])):
        temp = []
        for j in range(len(matrix)):
            temp.append(matrix[j][i])
        transposed.append(temp)

    transposed_dict = {}
    
    for key in range(1,len(transposed)+1):
        transposed_dict[key] = transposed[key-1]
    del transposed 
    del matrix
    return transposed_dict

In [337]:
def lsh(klasor_adi, k, b, r):
    import os
    import string
    import sys
    import random

    matrix_represent = dict() # shingle'larin matrixteki temsilini tutacak

    dosya_isimleri = os.listdir(klasor_adi) # klasorun icindeki tum dosyalarin adini aliyoruz

    # tüm dosyaları tek tek oku
    for dosya in dosya_isimleri: 
        content = open(klasor_adi + "/" + dosya, "r").read()
        content = content.replace("\n", " ")
        content = content.lower()
        content = content.translate(str.maketrans('', '', string.punctuation))
        content = " ".join(content.split())        
        content = content.split(" ")
        for i in range(len(content) - k):
            shingle =  content[i:i + k]
            shingle = " ".join(shingle)
            if shingle not in matrix_represent.keys():
                vec = zero_vec(len(dosya_isimleri))
                vec[dosya_isimleri.index(dosya)] = 1
                matrix_represent[shingle] = vec
            else:
                matrix_represent[shingle][dosya_isimleri.index(dosya)] = 1

    # shingle'larin sırasını karıştırıyoruz
    keys = list(matrix_represent.keys())
    random.shuffle(keys)

    temp_dict = {}
    for key in keys:
        temp_dict[key] = matrix_represent[key]
    del matrix_represent
    matrix_represent = temp_dict
    # ------------------------------

    num_of_shingles = len(matrix_represent.keys()) # shingle sayisi

    different_permutations = list() # farklı permütasyonları tutacak
    for i in range(10):
        temp = list(range(num_of_shingles))
        random.shuffle(temp)
        different_permutations.append(temp)
   

    signature_matrix = dict() # signature matrix oluşturuluyor
    transposed_matrix = transpose_dict(matrix_represent) # matrix_represent transpozu alınıyor

    for permutation in different_permutations: # her bir permütasyon için 
        i = different_permutations.index(permutation)
        print("Permutation: ", permutation)
        temp = []
        for j in transposed_matrix.keys(): # her bir doküman için
            for index in permutation: # her bir permütasyonun içindeki indexler için
                if transposed_matrix[j][index] == 1:
                    temp.append(index)
                    break 
            signature_matrix[i] = temp

    del different_permutations, temp_dict
    # kod buraya kadar çalışıyor

    # signature matrix'i b ve r değerlerine göre bölüyoruz
    signature_matrix_bolunmus = dict()
    for i in range(b): # signature matrix'i b'ye bölüyoruz
        signature_matrix_bolunmus[i] = []
    
        for doc in range(len(signature_matrix[0])): # her bir doküman için
            temp = []
            for j in range(r):
                temp.append(signature_matrix[i+j][doc])
            signature_matrix_bolunmus[i].append(temp)
    
    
    buckets = dict() # bucket'lar oluşturuluyor

    for i in range(b): # her bir b için
        buckets[i] = dict()
        for j in range(len(signature_matrix_bolunmus[i])): # her bir doküman için
            numbers = signature_matrix_bolunmus[i][j]
            concatenated_str = ''.join(str(num) for num in numbers)
            unicode_values = [ord(char) for char in concatenated_str]
            unicode_values_concatenated = ''.join(str(num) for num in unicode_values)
            if unicode_values_concatenated not in buckets[i].keys():
                buckets[i][unicode_values_concatenated] = [dosya_isimleri[j]]
            else:
                buckets[i][unicode_values_concatenated].append(dosya_isimleri[j])

    return buckets, signature_matrix_bolunmus




    

    

In [338]:
a,b = lsh("docs2", 2,5,2)

Permutation:  [1385, 2361, 2373, 4191, 2863, 3315, 821, 2114, 4166, 3004, 1558, 1226, 4135, 3242, 367, 3571, 743, 1542, 3231, 2797, 2394, 4242, 3658, 369, 3355, 3769, 4105, 3009, 3223, 994, 665, 3023, 2137, 1025, 753, 2141, 3686, 2068, 2779, 3194, 3034, 2560, 1400, 1976, 2909, 3121, 3493, 913, 835, 3519, 2205, 1793, 2726, 83, 1900, 51, 4209, 1961, 972, 2055, 2294, 1585, 3241, 2340, 2498, 160, 1374, 541, 528, 1234, 2605, 3096, 4273, 606, 521, 1767, 4176, 3995, 1358, 41, 4008, 2228, 3213, 1924, 3309, 941, 1978, 3817, 1604, 3127, 3554, 1040, 3883, 2211, 2555, 1996, 3169, 1746, 1246, 2712, 3413, 599, 1504, 2236, 314, 3043, 392, 2201, 1136, 3224, 982, 3557, 3888, 1469, 4115, 2418, 1292, 2308, 4238, 2987, 3873, 3400, 3246, 3842, 150, 1325, 3356, 1399, 2184, 3184, 1189, 1264, 2927, 193, 1520, 2328, 220, 379, 1470, 3618, 3867, 2133, 4161, 3233, 640, 2894, 1349, 2542, 4021, 2037, 2008, 1354, 374, 2241, 562, 3918, 2796, 3086, 3062, 3528, 1355, 1002, 551, 175, 4005, 4144, 3991, 1786, 3204, 2890, 

In [339]:
for key in a.keys():
    if len(a[key]) > 1:
        print(key, ":", a[key])

0 : {'5249574951515350': ['1000.txt'], '5056545149535155': ['1001.txt'], '50555755515054': ['1002.txt'], '5056545152505557': ['1003.txt', '1006.txt'], '50565451515754': ['1004.txt'], '5056545151485457': ['1005.txt'], '5056545151534951': ['1007.txt'], '5056545149535452': ['1008.txt'], '505154495650': ['1009.txt'], '5051555151534951': ['1010.txt'], '5056545150515549': ['1011.txt'], '5056545151565457': ['1012.txt'], '50515449495048': ['1013.txt'], '5053544850555052': ['1014.txt'], '5056545151505249': ['1015.txt'], '5055575551485457': ['1016.txt'], '5148485250545150': ['1017.txt'], '49515653495253': ['1018.txt'], '5055575550565749': ['1019.txt'], '50565451555448': ['1020.txt'], '51545550505154': ['1021.txt'], '5055575551485548': ['1022.txt'], '5249515349575553': ['1023.txt'], '5051575250555052': ['1024.txt'], '5151495350555052': ['1025.txt'], '51514953515056': ['1026.txt'], '5151495351534951': ['1027.txt'], '5150525050555052': ['1028.txt'], '5151495351485457': ['1029.txt']}
1 : {'515153505

In [340]:
a.keys()

dict_keys([0, 1, 2, 3, 4])

In [341]:
a[0]

{'5249574951515350': ['1000.txt'],
 '5056545149535155': ['1001.txt'],
 '50555755515054': ['1002.txt'],
 '5056545152505557': ['1003.txt', '1006.txt'],
 '50565451515754': ['1004.txt'],
 '5056545151485457': ['1005.txt'],
 '5056545151534951': ['1007.txt'],
 '5056545149535452': ['1008.txt'],
 '505154495650': ['1009.txt'],
 '5051555151534951': ['1010.txt'],
 '5056545150515549': ['1011.txt'],
 '5056545151565457': ['1012.txt'],
 '50515449495048': ['1013.txt'],
 '5053544850555052': ['1014.txt'],
 '5056545151505249': ['1015.txt'],
 '5055575551485457': ['1016.txt'],
 '5148485250545150': ['1017.txt'],
 '49515653495253': ['1018.txt'],
 '5055575550565749': ['1019.txt'],
 '50565451555448': ['1020.txt'],
 '51545550505154': ['1021.txt'],
 '5055575551485548': ['1022.txt'],
 '5249515349575553': ['1023.txt'],
 '5051575250555052': ['1024.txt'],
 '5151495350555052': ['1025.txt'],
 '51514953515056': ['1026.txt'],
 '5151495351534951': ['1027.txt'],
 '5150525050555052': ['1028.txt'],
 '5151495351485457': ['102

In [342]:
a[1]

{'5151535051495755': ['1000.txt'],
 '4953515550565349': ['1001.txt'],
 '51505449555155': ['1002.txt'],
 '52505557495349': ['1003.txt'],
 '51575451494952': ['1004.txt'],
 '5148545751495755': ['1005.txt'],
 '5250555750494855': ['1006.txt'],
 '51534951494948': ['1007.txt'],
 '495354524957': ['1008.txt'],
 '565051485248': ['1009.txt'],
 '5153495150495354': ['1010.txt'],
 '5051554950494855': ['1011.txt'],
 '5156545751495755': ['1012.txt'],
 '49504851485248': ['1013.txt'],
 '5055505249535350': ['1014.txt'],
 '5150524950574848': ['1015.txt'],
 '5148545751525556': ['1016.txt', '1029.txt'],
 '5054515049524855': ['1017.txt'],
 '49525350544953': ['1018.txt'],
 '5056574951495454': ['1019.txt'],
 '55544851485248': ['1020.txt'],
 '5050515451525556': ['1021.txt'],
 '5148554849505053': ['1022.txt'],
 '4957555351565556': ['1023.txt'],
 '5055505251525556': ['1024.txt'],
 '5055505250494855': ['1025.txt'],
 '51505652495354': ['1026.txt'],
 '5153495149535251': ['1027.txt'],
 '50555052515157': ['1028.txt']}

In [343]:
a[2]

{'5149575550535054': ['1000.txt', '1005.txt', '1012.txt'],
 '5056534950535054': ['1001.txt'],
 '4955515550535054': ['1002.txt'],
 '49534950535054': ['1003.txt'],
 '5149495250535054': ['1004.txt'],
 '5049485550535054': ['1006.txt', '1011.txt'],
 '49494850535054': ['1007.txt'],
 '495750535054': ['1008.txt'],
 '5148524850535054': ['1009.txt', '1013.txt', '1020.txt'],
 '5049535450535054': ['1010.txt'],
 '4953535050535054': ['1014.txt'],
 '5057484850535054': ['1015.txt'],
 '5152555650535054': ['1016.txt', '1021.txt', '1024.txt', '1029.txt'],
 '4952485550535054': ['1017.txt'],
 '5054495350535054': ['1018.txt'],
 '5149545450535054': ['1019.txt'],
 '4950505350535054': ['1022.txt'],
 '5156555650535054': ['1023.txt'],
 '5049485549': ['1025.txt'],
 '5249535450535054': ['1026.txt'],
 '4953525150535054': ['1027.txt'],
 '51515750515357': ['1028.txt']}

In [344]:
a[3]#]

{'50535054555348': ['1000.txt', '1029.txt'],
 '5053505449555553': ['1001.txt'],
 '50535054575356': ['1002.txt', '1006.txt'],
 '5053505451485057': ['1003.txt'],
 '5053505449525555': ['1004.txt', '1011.txt'],
 '5053505449535153': ['1005.txt'],
 '5053505452505452': ['1007.txt'],
 '5053505451535549': ['1008.txt'],
 '5053505450574954': ['1009.txt'],
 '5053505451575050': ['1010.txt'],
 '5053505450545057': ['1012.txt', '1016.txt'],
 '5053505450535555': ['1013.txt'],
 '5053505450504948': ['1014.txt'],
 '5053505450565249': ['1015.txt'],
 '5053505451515651': ['1017.txt'],
 '50535054535254': ['1018.txt', '1027.txt'],
 '50535054515754': ['1019.txt', '1022.txt'],
 '5053505450505552': ['1020.txt'],
 '5053505451495254': ['1021.txt'],
 '5053505449485249': ['1023.txt'],
 '5053505449495455': ['1024.txt'],
 '4950545148': ['1025.txt'],
 '5053505450514848': ['1026.txt'],
 '5051535749485055': ['1028.txt']}

In [345]:
a[4]

{'55534851485656': ['1000.txt'],
 '49555553575356': ['1001.txt'],
 '575356575356': ['1002.txt', '1006.txt'],
 '51485057565651': ['1003.txt'],
 '4952555549535652': ['1004.txt'],
 '4953515351485656': ['1005.txt'],
 '5250545249505455': ['1007.txt'],
 '5153554949564955': ['1008.txt'],
 '5057495456': ['1009.txt'],
 '5157505051505455': ['1010.txt'],
 '4952555549535350': ['1011.txt'],
 '5054505751555556': ['1012.txt', '1016.txt'],
 '50535555495654': ['1013.txt'],
 '5050494849535350': ['1014.txt'],
 '50565249575356': ['1015.txt'],
 '51515651514849': ['1017.txt'],
 '535254575356': ['1018.txt'],
 '51575449535350': ['1019.txt', '1022.txt'],
 '5050555256': ['1020.txt'],
 '5149525449485449': ['1021.txt'],
 '4948524949565149': ['1023.txt'],
 '4949545550575151': ['1024.txt'],
 '50545148535550': ['1025.txt'],
 '5051484850515350': ['1026.txt'],
 '53525451515048': ['1027.txt'],
 '49485055535550': ['1028.txt'],
 '55534850515350': ['1029.txt']}

Yukarıda aynı bucketta olanlar çalışıyor.

In [321]:
for i in b.keys():
    print(i, b[i])

0 [[511, 1012], [320, 1012], [243, 1012], [1775, 1012], [248, 1012], [991, 1012], [248, 1012], [865, 1012], [1587, 1012], [248, 1012], [248, 1012], [1587, 196], [243, 1012]]
1 [[1012, 1547], [1012, 1771], [1012, 1396], [1012, 1771], [1012, 1771], [1012, 1771], [1012, 1771], [1012, 117], [1012, 1771], [1012, 720], [1012, 1747], [196, 1771], [1012, 1771]]
2 [[1547, 927], [1771, 1703], [1396, 1703], [1771, 347], [1771, 1703], [1771, 1379], [1771, 1703], [117, 1703], [1771, 1703], [720, 1703], [1747, 1703], [1771, 1703], [1771, 1703]]
3 [[927, 1176], [1703, 1612], [1703, 1810], [347, 1232], [1703, 943], [1379, 1016], [1703, 1408], [1703, 1798], [1703, 1117], [1703, 230], [1703, 1798], [1703, 885], [1703, 1016]]
4 [[1176, 930], [1612, 119], [1810, 119], [1232, 1342], [943, 938], [1016, 938], [1408, 938], [1798, 802], [1117, 1277], [230, 819], [1798, 288], [885, 373], [1016, 719]]


**Soru 2)**  Önceki soruda hazırladığınız LSH indeksini kullanarak verilen bir dosyanın muhtemel benzerleri olan dosyaları ve o dokümanlar ile olan jaccard benzerliği sıralı bir şekilde ekrana yazdıran bir method yazınız.  En çok benzeyen dokümanlar ilk yazılmalı. Çıktının formatı aşağıdaki örnekteki gibi olmalı.  Örnekte sadece 2 dosya döndüğü varsayılmıştır.

a.txt 0.7
b.txt 0.65
Toplam 2 dosya bulunmuştur.


In [15]:
# Cevabınız buraya yazınız. Eğer ihtiyaç duyarsanız metoda başka parametreler ekleyebilirsiniz
def query(input_file):  ... 

**Soru 3)** Bu soruda ödev ile verilen  metin dosyalarını kullanarak LSH indeksi kurup farklı k, b, ve r değerlerinin sonuç üzerindeki etkisine bakacaksınız.
Spesifik olarak yapacağınız şunlar.
- Ödevde verilen docs klasöründeki dokümanları b=5, r=4 iken k=2,3,4 değerlerini kullanarak 3 farklı LSH indeksi kurunuz. Ardından ödevde verilen sorgu.txt dosyasını bu 3 ayrı indekste aratıp toplamda kaç doküman döndüğü gösteren bir 'bar chart' çiziniz. 
- k=3, r=4 iken b değerini 3,4,5 olarak değiştiriniz ve birinci şıktaki gibi toplamda kaç doküman döndüğü gösteren bir 'bar chart' çiziniz. 
- k=3, r=4 iken r değerini 3,4,5 olarak değiştiriniz ve birinci şıktaki gibi toplamda kaç doküman döndüğü gösteren bir 'bar chart' çiziniz.

Not:  Bu soruda ikinci sorudaki metotta ufak bir değişikliğe gidip ekrana bir şey bastırmadan sadece  kaç doküman bulunduğunu dönen bir metot olarak tanımlayın. Bu yeni metodu aşağıda 3. soru için tanımlanan yerde yapın.

In [16]:
# 3. sorunun cevabı için ayrılan yer

**Soru 4**	Bu soruda MeanShift algoritmasını implement edeceksiniz. Yazacağınız metot verilen klasördeki her dosyayı meanShift algoritmasına ve radius uzaklık parametresine göre kümeler ve çıktı olarak kümeleri print eder. Algoritma en başta klasör içindeki her dosya için bir ağırlık merkezi başlatır. Çıktısı şu formatta olmalıdır.

Cluster-0: a,b,c

Cluster-1: d,e,f 

...........

Cluster-n: h
 
Yukarıdaki örnekte a,b,c, vs. verilen klasördeki dosyaların isimleridir. Dokümanların vektörel gösterimi şöyle olmalıdır: Tüm dokümanlarda geçen en popüler 1000 kelimeyi bulun. Sonra her kelimenin varlığı/yokluğunu birer öznitelik olarak gösterin. Örneğin bir doküman “a b c d f” ise ve b, d ve s en popüler ‘kelimeler’ ise, bu dokümanın gösterimi [1,1,0] olmalıdır. Uzaklık için kosinüs uzaklığını kullanın.



In [17]:
def meanShift (String folder_name, float radius):

SyntaxError: invalid syntax (1091480408.py, line 1)

**Soru 5** 4. soruda tanımladığınız metodu kullanarak ödev ile beraber verilen docs klasöründeki dosyaları cluster ediniz. Bunun için radius değerinini 0.1’den başlayıp 0.05 arttırarak 0.95’e kadar tüm değerlerde deneyin ve elde ettiğiniz cluster sayısını gösteriniz. Bunun için x ekseninde Radius değeri, y ekseninde cluster sayısı olan bir figür oluşturunuz.